In [293]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn import neighbors
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Imputer

In [294]:
properties = pd.read_csv('/home/santiago/Desktop/Organizacion De Datos (75.06)/TP2/ClusteringTrain.csv', low_memory=False)
test_set = pd.read_csv('/home/santiago/Desktop/Organizacion De Datos (75.06)/TP2/testPreProcess.csv', low_memory=False)

In [295]:
all_df = test_set.append(properties)
all_df['lat'] = all_df['lat'].apply(lambda x: x*2000)
all_df['lon'] = all_df['lon'].apply(lambda x: x*2000)

properties['lat'] = properties['lat'].apply(lambda x: x*2000)
properties['lon'] = properties['lon'].apply(lambda x: x*2000)
len(all_df)
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134660 entries, 0 to 120493
Data columns (total 18 columns):
Unnamed: 0                  134660 non-null int64
Unnamed: 0.1                120494 non-null float64
created_on                  134660 non-null object
distancia_minima_privado    134660 non-null float64
distancia_minima_publico    134660 non-null float64
distancia_minima_subte      134660 non-null float64
distancia_obelisco          134660 non-null float64
floor                       134660 non-null float64
id                          134660 non-null object
lat                         134660 non-null float64
lon                         134660 non-null float64
place_name_encoded          134660 non-null int64
price                       120494 non-null float64
property_type_encoded       134660 non-null int64
rooms                       134660 non-null float64
state_name_encoded          134660 non-null int64
surface_covered_in_m2       134660 non-null float64
surface_total_i

In [296]:
#train = all_df.loc[:, ['property_type_encoded', 'place_name_encoded', 'rooms'
 #                       ,'surface_covered_in_m2', 'surface_total_in_m2',]]

#train = all_df.loc[:, ['property_type_encoded', 'rooms','lat','lon'
 #                       ,'surface_covered_in_m2']]
    
train = all_df.loc[:, ['place_name_encoded','surface_covered_in_m2','lat','lon','distancia_minima_subte']]


In [297]:
kmeans = KMeans(init='k-means++', n_clusters= 1800, n_init=1)#150 y 10
cluster_prediction = kmeans.fit(train).predict(train) 
cluster_prediction = cluster_prediction.flatten()
train['cluster'] = cluster_prediction

In [298]:
train_clusters = train['cluster'][len(test_set):]
test_clusters = train['cluster'][:len(test_set)]

#Agrego los clusters a los datasets
properties['cluster'] = train_clusters
test_set['cluster'] = test_clusters

In [299]:
#Ya termino K-means, ahora empieza KNN
train = properties.loc[:, ['place_name_encoded', 'cluster','surface_covered_in_m2',
                           'lat','lon','distancia_minima_subte', 'price']]

In [300]:
#test = train[int(0.88*len(train)):]
#train = train[:int(0.88*len(train))]

test = train #Comentar esta linea y descomentar las otras 2 para el intento
len(test)

120494

In [301]:
#train_no_prices = train.loc[:, ['property_type_encoded','surface_covered_in_m2','surface_total_in_m2','rooms',
 #                               'cluster']]

train_no_prices = train.loc[:, ['cluster']]

In [302]:
#test_to_predict = test.loc[:, ['property_type_encoded','surface_covered_in_m2','surface_total_in_m2'
 #                              ,'rooms','cluster']]


#real_test_to_predict = test_set.loc[:, ['property_type_encoded','surface_covered_in_m2','surface_total_in_m2'
 #                                       ,'rooms','cluster']]
    
test_to_predict = test.loc[:, ['cluster']]
real_test_to_predict = test_set.loc[:, ['cluster']]

In [303]:
knn = neighbors.KNeighborsRegressor(20, weights="distance", algorithm="ball_tree")

In [304]:
train_prices = train.loc[:, 'price']

In [305]:
predictions = knn.fit(train_no_prices, train_prices).predict(test_to_predict)
predictions = predictions.flatten()

real_predictions = knn.fit(train_no_prices, train_prices).predict(real_test_to_predict)
real_predictions = real_predictions.flatten()

In [306]:
result = pd.DataFrame()
result['price_usd'] = predictions
result['real_price'] = test['price'].values
result['dif'] = abs(result['real_price'] - result['price_usd'])
#result = result.sort_values('dif', ascending = False)

#El que sigue es el que va para submit
real_result = pd.DataFrame()
real_result['price_usd'] = real_predictions 
real_result['id'] = test_set['id'].values

real = test
score = mean_squared_error(result['real_price'].values, result['price_usd'])
score


14059162067.026415

In [307]:
result

,price_usd,real_price,dif
0,756000.000000,570000.0,186000.000000
1,756000.000000,500000.0,256000.000000
2,133200.000000,450000.0,316800.000000
3,129825.000000,135000.0,5175.000000
4,196650.000000,180000.0,16650.000000
5,137600.000000,77000.0,60600.000000
6,223650.000000,570000.0,346350.000000
7,528175.000000,98500.0,429675.000000
8,147700.000000,180000.0,32300.000000
9,422500.000000,150000.0,272500.000000


In [308]:
len(result[result['dif']>100000])

25892

In [309]:
real_result.index = real_result['id']
real_result = real_result[['price_usd']]
real_result.dropna(inplace=True)
real_result

,price_usd
id,
3632,756000.000000
3633,200273.535714
2263404,135963.250000
2263405,104259.400000
2263406,104259.400000
2263407,104259.400000
2263408,104259.400000
2263409,104259.400000
2263410,204471.050000


In [310]:
real_result.to_csv('K_Means + KNN 26-11.csv')

In [311]:
len(real_result)

14166